In [1]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as f
from pyspark.sql.types import *
import datetime
import dateutil.relativedelta
import os
import sys
import argparse

spark = SparkSession.builder \
.master("local") \
.appName("CS in HT") \
.getOrCreate()

In [2]:
cdp = spark.read.option("header",True).option("inferSchema",True).option("delimiter",";")\
   .csv("file:////home/abhijeet/spark coding/Data_sample/ACCOUNT_HISTORY_sample.csv")
cdp.printSchema()
cdp.registerTempTable('cs')
cdp.show()

root
 |-- ACCOUNT_ID: long (nullable = true)
 |-- REF_DATE: integer (nullable = true)
 |-- ACCOUNT_AMT: string (nullable = true)
 |-- OIB: string (nullable = true)

+------------+--------+-----------+-----------+
|  ACCOUNT_ID|REF_DATE|ACCOUNT_AMT|        OIB|
+------------+--------+-----------+-----------+
|  7216916006|  201701|     282,61| 8564252446|
|  5015855965|  201703|     301,74|93344157339|
|  5009136085|  201709|     437,68|78413880345|
|  5010856814|  201708|     137,24|99004437605|
|  9000005306|  201708|     122,42|35193478941|
|200003064142|  201708|        218|52022704995|
|  5007473192|  201705|     510,24|78124542987|
|200003460518|  201710|     220,12|70551125715|
|  5030225235|  201705|     168,71|47019207033|
|  9000104270|  201708|      50,04|35309054237|
|200002882837|  201709|        409|21630414169|
|  5012192877|  201701|     218,26| 4024361993|
|200001088378|  201703|      122,5|26497384513|
|  5007036142|  201701|      75,43|31043772224|
|200003018777|  201

In [ ]:
cdp_cnt=spark.sql("select distinct(ACCOUNT_ID) from cs")
cdp_cnt.count()


In [25]:
newDf = cdp.withColumn('ACCOUNT_AMT', f.regexp_replace('ACCOUNT_AMT', ',', '.'))
newDf.show()

+------------+--------+-----------+-----------+
|  ACCOUNT_ID|REF_DATE|ACCOUNT_AMT|        OIB|
+------------+--------+-----------+-----------+
|  7216916006|  201701|     282.61| 8564252446|
|  5015855965|  201703|     301.74|93344157339|
|  5009136085|  201709|     437.68|78413880345|
|  5010856814|  201708|     137.24|99004437605|
|  9000005306|  201708|     122.42|35193478941|
|200003064142|  201708|        218|52022704995|
|  5007473192|  201705|     510.24|78124542987|
|200003460518|  201710|     220.12|70551125715|
|  5030225235|  201705|     168.71|47019207033|
|  9000104270|  201708|      50.04|35309054237|
|200002882837|  201709|        409|21630414169|
|  5012192877|  201701|     218.26| 4024361993|
|200001088378|  201703|      122.5|26497384513|
|  5007036142|  201701|      75.43|31043772224|
|200003018777|  201712|      315.8|84810081508|
|100000078268|  201710|     4796.9|53889263424|
|200002934323|  201712|      324.7|50495590807|
|  5019361494|  201709|     100.63|20790

In [26]:
df=newDf.groupBy("ACCOUNT_ID","REF_DATE").agg((f.sum("ACCOUNT_AMT")).alias("Acc_Amt"))
df.show()

+------------+--------+-------+
|  ACCOUNT_ID|REF_DATE|Acc_Amt|
+------------+--------+-------+
|  5007858410|  201708| 317.43|
|  5007411572|  201711| 322.83|
|200003184110|  201708| 149.35|
|  5019287365|  201712|  90.25|
|200001494287|  201706|   3.65|
|  5005911861|  201706|  164.3|
|  5011293513|  201711|  403.0|
|  5007415224|  201707| 315.63|
|  5005162611|  201703|   95.0|
|  5007272159|  201702|  115.9|
|  5010043323|  201705|  289.5|
|  5018288077|  201708| 332.38|
|  5007374986|  201707|   95.2|
|  5011420726|  201711|   95.0|
|200002682109|  201703| 337.93|
|200001883945|  201701|   85.0|
|200003194962|  201704| 119.27|
|200001541271|  201701|  95.34|
|  5019594782|  201707|  243.5|
|  5005603891|  201707| 419.63|
+------------+--------+-------+
only showing top 20 rows



AnalysisException: "cannot resolve '`ACCOUNT_AMT`' given input columns: [ACCOUNT_ID, REF_DATE, OIB, OIB_Amt];;\n'Aggregate [sum('ACCOUNT_AMT) AS Acc_Amt#497]\n+- Aggregate [ACCOUNT_ID#10L, REF_DATE#11, OIB#13], [ACCOUNT_ID#10L, REF_DATE#11, OIB#13, sum(cast(ACCOUNT_AMT#440 as double)) AS OIB_Amt#491]\n   +- Project [ACCOUNT_ID#10L, REF_DATE#11, regexp_replace(ACCOUNT_AMT#12, ,, .) AS ACCOUNT_AMT#440, OIB#13]\n      +- Relation[ACCOUNT_ID#10L,REF_DATE#11,ACCOUNT_AMT#12,OIB#13] csv\n"

In [28]:
df1=newDf.groupBy("REF_DATE","OIB").agg((f.sum("ACCOUNT_AMT")).alias("OIB_Amt"))
df1.show()

+--------+-----------+-------+
|REF_DATE|        OIB|OIB_Amt|
+--------+-----------+-------+
|  201705| 4865071961|   95.0|
|  201706|81793654902|  130.0|
|  201701|77847917043| 109.05|
|  201709|34291906535| 369.92|
|  201708|75954059461| 229.59|
|  201704|62966585386|  214.5|
|  201711| 5820244871|  333.0|
|  201706|61945660682| 445.38|
|  201703|14634271695| 258.66|
|  201709|38228820129|  35.03|
|  201712|90507671771|  847.5|
|  201706|26567508317| 172.93|
|  201706|97174157946|  95.58|
|  201705|55895014490| 317.93|
|  201707|34145747388| 266.23|
|  201707|87511777652| 130.05|
|  201705|24364104782|  100.0|
|  201712|86301894481| 390.26|
|  201702|71464202194| 254.86|
|  201711|42857628694|  403.0|
+--------+-----------+-------+
only showing top 20 rows



In [30]:
join = df.join(df1,"REF_DATE")
print(join)
join.show()

DataFrame[REF_DATE: int, ACCOUNT_ID: bigint, Acc_Amt: double, OIB: string, OIB_Amt: double]
+--------+------------+-------+----------+-------+
|REF_DATE|  ACCOUNT_ID|Acc_Amt|       OIB|OIB_Amt|
+--------+------------+-------+----------+-------+
|  201705|  5006281779| 216.39|4865071961|   95.0|
|  201705|  5009570461|  56.58|4865071961|   95.0|
|  201705|200001131604|  60.95|4865071961|   95.0|
|  201705|  9000132823|  74.48|4865071961|   95.0|
|  201705|200000979825|   75.9|4865071961|   95.0|
|  201705|  5030261506|   90.0|4865071961|   95.0|
|  201705|  5016252801|   16.2|4865071961|   95.0|
|  201705|  5005143826|   95.0|4865071961|   95.0|
|  201705|  5006197205| 165.83|4865071961|   95.0|
|  201705|200003016555|  116.3|4865071961|   95.0|
|  201705|  5030077901| 174.86|4865071961|   95.0|
|  201705|  5007874942| 618.63|4865071961|   95.0|
|  201705|  9000126524|  85.12|4865071961|   95.0|
|  201705|  5012047250| 308.09|4865071961|   95.0|
|  201705|200001110444|  53.04|4865071961

In [33]:
pct =join.withColumn("ACCOUNT_PCT", (f.col("OIB_Amt") / f.col("Acc_Amt")))
pct.show()

+--------+------------+-------+----------+-------+-------------------+
|REF_DATE|  ACCOUNT_ID|Acc_Amt|       OIB|OIB_Amt|        ACCOUNT_PCT|
+--------+------------+-------+----------+-------+-------------------+
|  201705|  5006281779| 216.39|4865071961|   95.0| 0.4390221359582236|
|  201705|  5009570461|  56.58|4865071961|   95.0|   1.67903852951573|
|  201705|200001131604|  60.95|4865071961|   95.0| 1.5586546349466774|
|  201705|  9000132823|  74.48|4865071961|   95.0| 1.2755102040816326|
|  201705|200000979825|   75.9|4865071961|   95.0| 1.2516469038208167|
|  201705|  5030261506|   90.0|4865071961|   95.0| 1.0555555555555556|
|  201705|  5016252801|   16.2|4865071961|   95.0|  5.864197530864198|
|  201705|  5005143826|   95.0|4865071961|   95.0|                1.0|
|  201705|  5006197205| 165.83|4865071961|   95.0| 0.5728758367002351|
|  201705|200003016555|  116.3|4865071961|   95.0| 0.8168529664660361|
|  201705|  5030077901| 174.86|4865071961|   95.0| 0.5432917762781654|
|  201

In [ ]:
cdp = cdp.

In [ ]:
df=cdp.groupBy("Account_id",(str("Account_Amt")))

In [ ]:
df1=df.groupBy(["Account_Amt"]).count()

In [ ]:
df=cdp.groupBy("Account_id","Ref_date").agg((f.sum("Account_Amt")).alias("Amt"))
df.show()

In [ ]:
df1=df.select("Account_id"="7216","Ref_date","Amt").show()

In [ ]:
df1 = df1.withColumn("Amt", udf(lambda x: x+x, StringType())("Amt"))

In [ ]:
df2=df1.groupBy(("Account_id"),["Amt"]).sum()